In [15]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

import pickle

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge

from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score

In [16]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/Users/venicius/Documents/codes/mlops-zoomcamp/01-intro/mlruns/2', creation_time=1755220812333, experiment_id='2', last_update_time=1755220812333, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [17]:
def read_dataframe(filename = './data/green_tripdata_2021-01.parquet'):
    df= pd.read_parquet(filename)

    df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])
    df['lpep_dropoff_datetime'] = pd.to_datetime(df['lpep_dropoff_datetime'])
    
    df['duration'] = df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']
    df['duration'] = df.duration.apply(lambda td: td.total_seconds()/60)
    df = df[(df.duration >= 1)&(df.duration <= 60)].copy()
    
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [18]:
df_train = read_dataframe()
df_val = read_dataframe(filename = './data/green_tripdata_2021-02.parquet')

In [19]:
df_train['PU_DU'] = df_train['PULocationID'] + '_'+ df_train['DOLocationID'] 
df_val['PU_DU'] = df_val['PULocationID'] + '_'+ df_val['DOLocationID'] 

In [20]:
categorical = ['PU_DU']#'PULocationID', 'DOLocationID']
numerical = ['trip_distance', ]

dv = DictVectorizer()

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [21]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [22]:
from sklearn.metrics import root_mean_squared_error


with mlflow.start_run():

    mlflow.set_tag("model", "linear-regression")
    mlflow.set_tag("developer", "venicius")
    mlflow.log_param("train-data-path", './data/green_tripdata_2021-01.parquet')
    mlflow.log_param("val-data-path", './data/green_tripdata_2021-02.parquet')
    mlflow.log_param("categorical", categorical)
    mlflow.log_param("numerical", numerical)  

    lr = LinearRegression()
    lr.fit(X_train, y_train)
    print('train')
    y_train_hat = lr.predict(X_train)

    print("RMSE", root_mean_squared_error(y_train, y_train_hat))#, squared = False))

    print("R2", r2_score(y_train, y_train_hat))
    print('val')
    y_val_hat = lr.predict(X_val)

    print("RMSE", root_mean_squared_error(y_val, y_val_hat))##, squared = False))

    print("R2", r2_score(y_val, y_val_hat))

    mlflow.log_metrics({
    "train_rmse": root_mean_squared_error(y_train, y_train_hat),
    "train_r2": r2_score(y_train, y_train_hat),
    "val_rmse": root_mean_squared_error(y_val, y_val_hat),
    "val_r2": r2_score(y_val, y_val_hat)
    })

train
RMSE 5.699564185837764
R2 0.7570393866219574
val
RMSE 7.758714882773319
R2 0.5937204490448376


with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)
    

In [23]:
with mlflow.start_run():
    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    mlflow.set_tag("model", "lasso-regression")
    mlflow.set_tag("developer", "venicius")
    mlflow.log_param("train-data-path", './data/green_tripdata_2021-01.parquet')
    mlflow.log_param("val-data-path", './data/green_tripdata_2021-02.parquet')
    mlflow.log_param("categorical", categorical)
    mlflow.log_param("numerical", numerical) 
    lr = Lasso(alpha=alpha, random_state=42)
    lr.fit(X_train, y_train)
    print('train')
    y_train_hat = lr.predict(X_train)

    print("RMSE", root_mean_squared_error(y_train, y_train_hat))#, squared = False))

    print("R2", r2_score(y_train, y_train_hat))
    print('val')
    y_val_hat = lr.predict(X_val)

    print("RMSE", root_mean_squared_error(y_val, y_val_hat))##, squared = False))

    print("R2", r2_score(y_val, y_val_hat))

    mlflow.log_metrics({
    "train_rmse": root_mean_squared_error(y_train, y_train_hat),
    "train_r2": r2_score(y_train, y_train_hat),
    "val_rmse": root_mean_squared_error(y_val, y_val_hat),
    "val_r2": r2_score(y_val, y_val_hat)
    })

train
RMSE 11.496877490906606
R2 0.011418714196373858
val
RMSE 12.143233939842847
R2 0.004791442007301638


with open('models/lasso.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)
    

In [24]:
with mlflow.start_run():
    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    mlflow.set_tag("model", "ridge-regression")
    mlflow.set_tag("developer", "venicius")
    mlflow.log_param("train-data-path", './data/green_tripdata_2021-01.parquet')
    mlflow.log_param("val-data-path", './data/green_tripdata_2021-02.parquet')
    mlflow.log_param("categorical", categorical)
    mlflow.log_param("numerical", numerical) 
   

    lr = Ridge(alpha=alpha, random_state=42)
    lr.fit(X_train, y_train)
    print('train')
    y_train_hat = lr.predict(X_train)

    print("RMSE", root_mean_squared_error(y_train, y_train_hat))#, squared = False))

    print("R2", r2_score(y_train, y_train_hat))
    print('val')
    y_val_hat = lr.predict(X_val)

    print("RMSE", root_mean_squared_error(y_val, y_val_hat))##, squared = False))

    print("R2", r2_score(y_val, y_val_hat))

    mlflow.log_metrics({
    "train_rmse": root_mean_squared_error(y_train, y_train_hat),
    "train_r2": r2_score(y_train, y_train_hat),
    "val_rmse": root_mean_squared_error(y_val, y_val_hat),
    "val_r2": r2_score(y_val, y_val_hat)
    })

train
RMSE 5.065771146648109
R2 0.8080696259439102
val
RMSE 7.5248930084504515
R2 0.6178392882936871


with open('models/ridge.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)
    

In [ ]:

train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)